In [35]:
%reload_ext autoreload
%autoreload 2
from src.acnets.pipeline import Parcellation
from src.acnets.pipeline import TimeseriesAggregator, ConnectivityExtractor
from src.acnets.pipeline import ConnectivityAggregator
import pandas as pd
import numpy as np

In [36]:
bids_dir = 'data/julia2018'
parcellation_cache_dir = 'data/julia2018/derivatives/resting_timeseries/'

# parcellation
parcellation = Parcellation(
    atlas_name='dosenbach2010',
    bids_dir=bids_dir,
    cache_dir=parcellation_cache_dir)
dataset = parcellation.fit_transform(None)

# time-series aggregation
ts_agg = TimeseriesAggregator(strategy=None)
dataset = ts_agg.fit_transform(dataset)

# connectivity extraction
conn_extractor = ConnectivityExtractor(kind='correlation')
dataset = conn_extractor.fit_transform(dataset)

# connectivity aggregation
conn_agg = ConnectivityAggregator(strategy='network')
dataset = conn_agg.fit_transform(dataset)

dataset

<xarray.Dataset>
Dimensions:               (timepoint: 124, region: 160, subject: 32,
                           region_src: 160, region_dst: 160, network_src: 6,
                           network_dst: 6)
Coordinates:
  * timepoint             (timepoint) int64 1 2 3 4 5 6 ... 120 121 122 123 124
  * region                (region) object 'vmPFC 1' ... 'post occipital 160'
  * subject               (subject) object 'AVGP01' 'AVGP02' ... 'NVGP19NEW'
    network               (region) object 'default' ... 'occipital'
  * region_src            (region_src) object 'vmPFC 1' ... 'post occipital 160'
  * region_dst            (region_dst) object 'vmPFC 1' ... 'post occipital 160'
  * network_src           (network_src) object 'cerebellum' ... 'sensorimotor'
  * network_dst           (network_dst) object 'cerebellum' ... 'sensorimotor'
Data variables: (12/13)
    timeseries            (subject, timepoint, region) float64 259.0 ... 601.1
    age                   (subject) float64 ...
    sex                   (subject) object ...
    group                 (subject) object ...
    preprocessed_beh      (subject) float64 ...
    preprocessed_rsfmri   (subject) float64 ...
    ...                    ...
    x                     (region) int64 ...
    y                     (region) int64 ...
    z                     (region) int64 ...
    index                 (region) int64 ...
    connectivity          (subject, region_src, region_dst) float64 1.0 ... 1.0
    network_connectivity  (subject, network_src, network_dst) float64 0.5295 ...